In [1]:
import mlflow
import mlflow.keras
import numpy as np
import pandas as pd
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.metrics import Recall, AUC
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import itertools
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve, auc, precision_score, recall_score, f1_score, accuracy_score, confusion_matrix
from tensorflow.keras.callbacks import EarlyStopping
import mlflow.keras
from mlflow.models import infer_signature

In [3]:
metadata = pd.read_csv('dataset/balanced_metadata.csv')
image_folder = 'dataset/balanced_dataset/'
images = []
labels = []
for i, row in metadata.iterrows():
    img_path = os.path.join(image_folder, row['image_name'])
    img = load_img(img_path, target_size=(128, 128))
    images.append(img_to_array(img))
    labels.append(row['target'])

images = np.array(images) / 255.0  # Normalisation
labels = np.array(labels)
print(labels.shape)
print(labels[:10])

(8000,)
[3 3 3 3 3 3 3 3 3 3]


In [4]:
labels = to_categorical(labels, num_classes=4)
print(images.shape)  
print(labels.shape)

(8000, 128, 128, 3)
(8000, 4)


In [5]:
def build_cnn_model(input_shape=(128, 128, 3), num_classes=4, optimizer='adam',
                    dropout_rate=0.5, activation='relu', filters=32, kernel_size=(3, 3)):
    model = Sequential()

    # Première couche convolutionnelle
    model.add(Conv2D(filters, kernel_size, activation=activation, input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))

    # Deuxième couche convolutionnelle
    model.add(Conv2D(filters * 2, kernel_size, activation=activation))
    model.add(MaxPooling2D((2, 2)))

    # Troisième couche convolutionnelle
    model.add(Conv2D(filters * 4, kernel_size, activation=activation))
    model.add(MaxPooling2D((2, 2)))

    # Aplatir les résultats des couches précédentes
    model.add(Flatten())

    # Couches entièrement connectées
    model.add(Dense(128, activation=activation))
    model.add(Dropout(dropout_rate))  # Dropout pour éviter l'overfitting
    model.add(Dense(num_classes, activation='softmax'))  # Classification multi-classe

    # Choisir l'optimiseur basé sur l'argument
    if optimizer == 'adam':
        optimizer_instance = Adam()
    elif optimizer == 'sgd':
        optimizer_instance = SGD()
    elif optimizer == 'rmsprop':
        optimizer_instance = RMSprop()

    # Compiler le modèle avec les métriques supplémentaires
    model.compile(optimizer=optimizer_instance, 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy', Recall(), AUC()])

    return model

In [ ]:
# Choix des paramètres
param_grid = {
    'optimizer': 'adam',  # Optimiseur à tester
    'dropout_rate': [0.3, 0.5],  # Taux de dropout
    'activation': 'relu',  # Fonction d'activation des couches
    'filters': [16, 32],  # Nombre de filtres dans les couches convolutionnelles
    'kernel_size': (3, 3),  # Taille des noyaux de convolution
    'batch_size': 32,  # Taille des mini-batchs
    'epochs': 10  # Nombre d'époques
}

In [7]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

- run on terminal **mlflow server --host 127.0.0.1 --port 5000**

In [8]:
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

- run on terminal **mlflow ui**

In [9]:
mlflow.set_experiment("Brain_Tumor_Classification")

<Experiment: artifact_location='mlflow-artifacts:/344762819335167390', creation_time=1733865490229, experiment_id='344762819335167390', last_update_time=1733865490229, lifecycle_stage='active', name='Brain_Tumor_Classification', tags={}>

In [10]:
experiment = mlflow.get_experiment_by_name("Brain_Tumor_Classification")
experiment_id = experiment.experiment_id

print(f"Experiment ID: {experiment_id}")

Experiment ID: 344762819335167390


In [11]:
# Paramètres pour la validation croisée
num_folds = 3
cv = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

In [ ]:
results = []
nb = 0
for filters in param_grid['filters']:
        for dropout_rate in param_grid['dropout_rate']:
                    nb = nb + 1

                    # Paramètres du modèle
                    params = {
                        'filters': filters,
                        'kernel_size': param_grid['kernel_size'],
                        'dropout_rate': dropout_rate,
                        'batch_size': param_grid['batch_size'],
                        'epochs': param_grid['epochs'],
                        'optimizer': param_grid['optimizer']
                    }
                    print(f"Starting experiment {nb} with parameters: {params}")

                    with mlflow.start_run() as run:
                        mlflow.set_tag("mlflow.runName", f"CNN : Experiment {nb}")
                        mlflow.set_tag("Experiment Info", f"Experiment {nb} for NN with parameters : {params}")
                        mlflow.log_params(params)

                        fold_metrics = []
                        for fold, (train_idx, val_idx) in enumerate(cv.split(X_train, np.argmax(y_train, axis=1))):
                            print(f"Fold {fold + 1}/{num_folds}")
                            
                            X_fold_train, X_fold_val = X_train[train_idx], X_train[val_idx]
                            y_fold_train, y_fold_val = y_train[train_idx], y_train[val_idx]

                            model = build_cnn_model(
                                input_shape=(128, 128, 3),
                                num_classes=4,
                                optimizer=params['optimizer'],
                                dropout_rate=params['dropout_rate'],
                                activation='relu',
                                filters=params['filters'],
                                kernel_size=params['kernel_size']
                            )
                            early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

                            history = model.fit(
                                X_fold_train, y_fold_train,
                                validation_data=(X_fold_val, y_fold_val),
                                batch_size=params['batch_size'],
                                epochs=params['epochs'],
                                callbacks=[early_stopping],
                                verbose=1
                            )

                            # Prédictions et métriques sur le fold
                            y_val_pred = model.predict(X_fold_val)
                            y_val_pred_classes = np.argmax(y_val_pred, axis=1)
                            y_val_actual_classes = np.argmax(y_fold_val, axis=1)

                            accuracy = accuracy_score(y_val_actual_classes, y_val_pred_classes)
                            precision = precision_score(y_val_actual_classes, y_val_pred_classes, average='weighted')
                            recall = recall_score(y_val_actual_classes, y_val_pred_classes, average='weighted')
                            f1 = f1_score(y_val_actual_classes, y_val_pred_classes, average='weighted')
                            auc_roc = roc_auc_score(y_fold_val, y_val_pred, multi_class='ovr')

                            fold_metrics.append({
                                'accuracy': accuracy,
                                'precision': precision,
                                'recall': recall,
                                'f1': f1,
                                'auc_roc': auc_roc
                            })

                            # Logging des métriques pour chaque fold
                            mlflow.log_metric(f"fold_{fold + 1}_accuracy", accuracy)
                            mlflow.log_metric(f"fold_{fold + 1}_precision", precision)
                            mlflow.log_metric(f"fold_{fold + 1}_recall", recall)
                            mlflow.log_metric(f"fold_{fold + 1}_f1", f1)
                            mlflow.log_metric(f"fold_{fold + 1}_auc_roc", auc_roc)
                           # Moyennes des métriques sur tous les folds
                        avg_accuracy = np.mean([m['accuracy'] for m in fold_metrics])
                        avg_precision = np.mean([m['precision'] for m in fold_metrics])
                        avg_recall = np.mean([m['recall'] for m in fold_metrics])
                        avg_f1 = np.mean([m['f1'] for m in fold_metrics])
                        avg_auc_roc = np.mean([m['auc_roc'] for m in fold_metrics])

                        mlflow.log_metric("avg_accuracy", avg_accuracy)
                        mlflow.log_metric("avg_precision", avg_precision)
                        mlflow.log_metric("avg_recall", avg_recall)
                        mlflow.log_metric("avg_f1", avg_f1)
                        mlflow.log_metric("avg_auc_roc", avg_auc_roc)

                        # Prédictions finales sur X_test
                        y_test_pred = model.predict(X_test)
                        y_test_pred_classes = np.argmax(y_test_pred, axis=1)
                        y_test_actual_classes = np.argmax(y_test, axis=1)

                        # Calcul des métriques sur le test
                        test_accuracy = accuracy_score(y_test_actual_classes, y_test_pred_classes)
                        test_precision = precision_score(y_test_actual_classes, y_test_pred_classes, average='weighted')
                        test_recall = recall_score(y_test_actual_classes, y_test_pred_classes, average='weighted')
                        test_f1 = f1_score(y_test_actual_classes, y_test_pred_classes, average='weighted')

                        mlflow.log_metric("test_accuracy", test_accuracy)
                        mlflow.log_metric("test_precision", test_precision)
                        mlflow.log_metric("test_recall", test_recall)
                        mlflow.log_metric("test_f1", test_f1)

                        predictions_df = pd.DataFrame({"Actual" : y_test_actual_classes, "Predicted": y_test_pred_classes})
                        predictions_csv_path = "tmp/predictions.csv"
                        predictions_df.to_csv(predictions_csv_path, index=False)
                        mlflow.log_artifact(predictions_csv_path, artifact_path="predictions")

                        # Matrice de confusion
                        matrix = confusion_matrix(y_test_actual_classes, y_test_pred_classes)
                        plt.figure(figsize=(8, 6))
                        sns.heatmap(matrix, annot=True, fmt='d', cmap='Blues')
                        plt.title("Confusion Matrix")
                        cf_matrix_path = "tmp/confusion_matrix.png"
                        plt.savefig(cf_matrix_path)
                        plt.close()
                        mlflow.log_artifact(cf_matrix_path, artifact_path="cf_matrix")

                        # Courbe ROC AUC
                        fpr, tpr, _ = roc_curve(y_test.ravel(), y_test_pred.ravel())
                        roc_auc = auc(fpr, tpr)

                        plt.figure()
                        plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
                        plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')
                        plt.xlabel("False Positive Rate")
                        plt.ylabel("True Positive Rate")
                        plt.title("Receiver Operating Characteristic")
                        plt.legend(loc="lower right")
                        roc_curve_path = "tmp/roc_curve.png"
                        plt.savefig(roc_curve_path)
                        plt.close()
                        mlflow.log_artifact(roc_curve_path, artifact_path="roc_curve")

                        # Enregistrement du modèle final
                        mlflow.keras.log_model(model, "cnn_model")
                        signature = infer_signature(X_train, model.predict(X_train))
                        model_info = mlflow.sklearn.log_model(
                            sk_model=model,
                            artifact_path="cnn_model",
                            signature=signature,
                            input_example=X_train[:1],
                            registered_model_name="convolutional-neural-network",
                        )


Starting experiment 1 with parameters: {'filters': 16, 'kernel_size': (5, 5), 'dropout_rate': 0.3, 'batch_size': 32, 'epochs': 10, 'optimizer': 'adam'}
Fold 1/3


c:\Users\user\Documents\GitHub\ML-Excuses\env\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 11s 92ms/step - accuracy: 0.4648 - auc: 0.7466 - loss: 1.1354 - recall: 0.2081 - val_accuracy: 0.6749 - val_auc: 0.8819 - val_loss: 0.8394 - val_recall: 0.5975
Epoch 2/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 10s 92ms/step - accuracy: 0.7185 - auc: 0.9174 - loss: 0.6870 - recall: 0.6504 - val_accuracy: 0.7668 - val_auc: 0.9446 - val_loss: 0.5636 - val_recall: 0.7422
Epoch 3/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 11s 106ms/step - accuracy: 0.8136 - auc: 0.9592 - loss: 0.4786 - recall: 0.7838 - val_accuracy: 0.8149 - val_auc: 0.9605 - val_loss: 0.4682 - val_recall: 0.7944
Epoch 4/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 10s 98ms/step - accuracy: 0.8321 - auc: 0.9708 - loss: 0.4012 - recall: 0.8176 - val_accuracy: 0.8582 - val_auc: 0.9732 - val_loss: 0.3780 - val_recall: 0.8453
Epoch 5/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 11s 99ms/step - accuracy: 0.8901 - auc: 0.9841 - loss: 0.2964 - recall: 0.8750 - val_accuracy: 0.8770 - val_auc: 0.9801 - val_loss: 0.3245 - val_recall: 0.8682

c:\Users\user\Documents\GitHub\ML-Excuses\env\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 12s 105ms/step - accuracy: 0.5288 - auc_1: 0.7869 - loss: 1.0616 - recall_1: 0.2954 - val_accuracy: 0.7680 - val_auc_1: 0.9383 - val_loss: 0.6144 - val_recall_1: 0.6749
Epoch 2/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 11s 104ms/step - accuracy: 0.7642 - auc_1: 0.9344 - loss: 0.6153 - recall_1: 0.6983 - val_accuracy: 0.7844 - val_auc_1: 0.9451 - val_loss: 0.5580 - val_recall_1: 0.7581
Epoch 3/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 11s 102ms/step - accuracy: 0.8103 - auc_1: 0.9626 - loss: 0.4611 - recall_1: 0.7856 - val_accuracy: 0.7803 - val_auc_1: 0.9527 - val_loss: 0.5171 - val_recall_1: 0.7545
Epoch 4/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 11s 102ms/step - accuracy: 0.8800 - auc_1: 0.9804 - loss: 0.3256 - recall_1: 0.8674 - val_accuracy: 0.8705 - val_auc_1: 0.9759 - val_loss: 0.3680 - val_recall_1: 0.8629
Epoch 5/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 11s 103ms/step - accuracy: 0.9096 - auc_1: 0.9899 - loss: 0.2343 - recall_1: 0.8995 - val_accuracy: 0.8799 - val_auc_1: 0.97

c:\Users\user\Documents\GitHub\ML-Excuses\env\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


107/107 ━━━━━━━━━━━━━━━━━━━━ 13s 113ms/step - accuracy: 0.4840 - auc_2: 0.7587 - loss: 1.1114 - recall_2: 0.2403 - val_accuracy: 0.7128 - val_auc_2: 0.9185 - val_loss: 0.6911 - val_recall_2: 0.6342
Epoch 2/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 12s 109ms/step - accuracy: 0.7417 - auc_2: 0.9280 - loss: 0.6448 - recall_2: 0.6751 - val_accuracy: 0.7989 - val_auc_2: 0.9528 - val_loss: 0.5520 - val_recall_2: 0.7239
Epoch 3/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 12s 114ms/step - accuracy: 0.7895 - auc_2: 0.9499 - loss: 0.5334 - recall_2: 0.7490 - val_accuracy: 0.8118 - val_auc_2: 0.9621 - val_loss: 0.4632 - val_recall_2: 0.7872
Epoch 4/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 12s 113ms/step - accuracy: 0.8388 - auc_2: 0.9705 - loss: 0.4027 - recall_2: 0.8191 - val_accuracy: 0.8306 - val_auc_2: 0.9654 - val_loss: 0.4453 - val_recall_2: 0.8154
Epoch 5/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 12s 114ms/step - accuracy: 0.8752 - auc_2: 0.9790 - loss: 0.3388 - recall_2: 0.8558 - val_accuracy: 0.8675 - val_auc_2: 0.9745 - val_lo

2024/12/10 23:06:42 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/12/10 23:06:48 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


160/160 ━━━━━━━━━━━━━━━━━━━━ 6s 38ms/step
